In [97]:
import pandas as pd
import requests
import time

from datetime import datetime, timedelta


# 결과를 저장할 빈 DataFrame 생성
combined_data = pd.DataFrame()


for _ in range(6):
    # 요청할 URL
    url = 'http://openapi.seoul.go.kr:8088/7942675a74636a75333361454b4358/json/bikeList/1/1000/'

    # GET 요청 보내기
    response = requests.get(url)

    # 응답 확인
    if response.status_code == 200:
        # JSON 데이터 추출
        data = response.json()
        
        # DataFrame으로 변환
        bike_data = pd.DataFrame(data['rentBikeStatus']['row'])
        
        # 중복 제거하고 새로운 데이터만 추가
        combined_data = pd.concat([combined_data, bike_data]).drop_duplicates().reset_index(drop=True)
    else:
        print('요청 실패:', response.status_code)
        
       
    
    url2 = 'http://openapi.seoul.go.kr:8088/7942675a74636a75333361454b4358/json/bikeList/1001/2000/'
    # GET 요청 보내기
    response = requests.get(url2)

    # 응답 확인
    if response.status_code == 200:
        # JSON 데이터 추출
        data = response.json()
        
        # DataFrame으로 변환
        bike_data = pd.DataFrame(data['rentBikeStatus']['row'])
        
        # 중복 제거하고 새로운 데이터만 추가
        combined_data = pd.concat([combined_data, bike_data]).drop_duplicates().reset_index(drop=True)
    else:
        print('요청 실패:', response.status_code)
        
        
    url3 = 'http://openapi.seoul.go.kr:8088/7942675a74636a75333361454b4358/json/bikeList/2001/2715/'
    # GET 요청 보내기
    response = requests.get(url3)

    # 응답 확인
    if response.status_code == 200:
        # JSON 데이터 추출
        data = response.json()
        
        # DataFrame으로 변환
        bike_data = pd.DataFrame(data['rentBikeStatus']['row'])
        
        # 중복 제거하고 새로운 데이터만 추가
        combined_data = pd.concat([combined_data, bike_data]).drop_duplicates().reset_index(drop=True)
    else:
        print('요청 실패:', response.status_code)
        
        
    
    # 5분 대기
    time.sleep(600)  # 600초(10분) 대기

print(combined_data)




     rackTotCnt      stationName parkingBikeTotCnt shared stationLatitude  \
0            15  102. 망원역 1번출구 앞                25    167     37.55564880   
1            14  103. 망원역 2번출구 앞                13     93     37.55495071   
2            13  104. 합정역 1번출구 앞                23    177     37.55062866   
3             5  105. 합정역 5번출구 앞                16    320     37.55000687   
4            12  106. 합정역 7번출구 앞                 0      0     37.54864502   
...         ...              ...               ...    ...             ...   
6229         10   5777. 호반써밋12단지                 5     50     37.48448563   
6230         15    5857. U레지던스 앞                19    127     37.51382446   
6231         10   5858. 영등포역5번출구                75    750     37.51669312   
6232         10    5860. 어울림병원 앞                 9     90     37.51350021   
6233         10      5867. H타워 앞                17    170     37.52024078   

     stationLongitude stationId  
0        126.91062927      ST-4  
1      

In [98]:
# 중복된 값 확인
duplicated_rows = combined_data.duplicated('stationId')
print('중복된 행의 수:', duplicated_rows.sum())

중복된 행의 수: 3519


In [99]:
# stationid로 묶은 다음에 총 개수, 남은 자전거 수는 적은걸로
combined_data = combined_data.groupby('stationId').agg({
    'parkingBikeTotCnt': min,
    'stationId': 'count'
}).rename(columns={'stationId': 'sum'}).reset_index()
combined_data

,stationId,parkingBikeTotCnt,sum
0,ST-10,5,3
1,ST-1000,10,4
2,ST-1002,54,5
3,ST-1003,6,3
4,ST-1004,4,3
...,...,...,...
2710,ST-993,4,2
2711,ST-994,39,2
2712,ST-995,3,1
2713,ST-996,14,1


In [100]:
with pd.option_context('display.max_rows', None):
    print(combined_data)

     stationId parkingBikeTotCnt  sum
0        ST-10                 5    3
1      ST-1000                10    4
2      ST-1002                54    5
3      ST-1003                 6    3
4      ST-1004                 4    3
5      ST-1005                29    3
6      ST-1006                16    3
7      ST-1007                10    2
8      ST-1008                 3    2
9      ST-1010                10    3
10     ST-1011                41    1
11     ST-1012                16    1
12     ST-1013                 1    2
13     ST-1014                20    4
14     ST-1015                 3    3
15     ST-1016                 2    2
16     ST-1017                25    6
17     ST-1018                10    4
18     ST-1019                38    3
19      ST-102                 6    3
20     ST-1020                 9    1
21     ST-1023                 5    1
22     ST-1024                17    1
23     ST-1025                 6    2
24     ST-1027                 0    1
25     ST-10

In [101]:
import pandas as pd

df2 = pd.read_csv('master_info_with_nearby.csv')
df2

,stn_id,stn_addr,stn_lat,stn_lng,nearby_id,nearby_km,stn_gu
0,ST-997,서울특별시 양천구 목동중앙로 49,37.534390,126.869598,ST-310,0.219206,양천구
1,ST-996,서울특별시 양천구 남부순환로88길5-16,37.524334,126.850548,ST-1008,0.218592,양천구
2,ST-995,서울특별시 양천구 중앙로 153 공중화장실,37.510597,126.857323,ST-1504,0.162262,양천구
3,ST-994,서울특별시 양천구 목동서로161,37.529163,126.872749,ST-427,0.092895,양천구
4,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,37.521511,126.857384,ST-2761,0.175407,양천구
...,...,...,...,...,...,...,...
3256,ST-1060,서울특별시 둔촌동 산 125,37.531390,127.150740,ST-1764,0.131280,둔촌동
3257,ST-1030,서울특별시 은평구 녹번동 141-92,37.607833,126.925130,ST-2784,0.272480,은평구
3258,ST-1022,서울특별시 진관동 2-6,37.650580,126.935820,ST-452,0.497978,진관동
3259,ST-1009,서울특별시 양천구 신월동 263,37.536548,126.828931,ST-1010,0.107213,양천구


In [102]:
#가까운 정류장 업데이트, address 열 날리기
updated_df = pd.merge(combined_data, df2[['stn_id', 'stn_lat', 'stn_lng', 'nearby_id']], left_on='stationId', right_on='stn_id', how='left')
updated_df = updated_df.drop(columns=['stn_id'])

print(updated_df)

     stationId parkingBikeTotCnt  sum    stn_lat     stn_lng nearby_id
0        ST-10                 5    3  37.552746  126.918617   ST-2167
1      ST-1000                10    4  37.510380  126.866798   ST-2014
2      ST-1002                54    5  37.529900  126.876541   ST-1505
3      ST-1003                 6    3  37.539551  126.828300   ST-2794
4      ST-1004                 4    3  37.514099  126.831001   ST-1498
...        ...               ...  ...        ...         ...       ...
2710    ST-993                 4    2  37.521511  126.857384   ST-2761
2711    ST-994                39    2  37.529163  126.872749    ST-427
2712    ST-995                 3    1  37.510597  126.857323   ST-1504
2713    ST-996                14    1  37.524334  126.850548   ST-1008
2714    ST-997                 4    3  37.534390  126.869598    ST-310

[2715 rows x 6 columns]


In [103]:
# 'stationId'와 'sum' 열만을 가진 새로운 DataFrame 생성
sum_df = updated_df[['stationId', 'sum']].copy()

# 'stationId'를 'nearby_id'로 이름 변경
sum_df.rename(columns={'stationId': 'nearby_id', 'sum': 'nearby_id\'s sum'}, inplace=True)

# 'nearby_id'를 기준으로 병합
updated_df = pd.merge(updated_df, sum_df, on='nearby_id', how='left')
print(updated_df)

     stationId parkingBikeTotCnt  sum    stn_lat     stn_lng nearby_id  \
0        ST-10                 5    3  37.552746  126.918617   ST-2167   
1      ST-1000                10    4  37.510380  126.866798   ST-2014   
2      ST-1002                54    5  37.529900  126.876541   ST-1505   
3      ST-1003                 6    3  37.539551  126.828300   ST-2794   
4      ST-1004                 4    3  37.514099  126.831001   ST-1498   
...        ...               ...  ...        ...         ...       ...   
2710    ST-993                 4    2  37.521511  126.857384   ST-2761   
2711    ST-994                39    2  37.529163  126.872749    ST-427   
2712    ST-995                 3    1  37.510597  126.857323   ST-1504   
2713    ST-996                14    1  37.524334  126.850548   ST-1008   
2714    ST-997                 4    3  37.534390  126.869598    ST-310   

      nearby_id's sum  
0                 2.0  
1                 4.0  
2                 NaN  
3              

In [104]:
with pd.option_context('display.max_rows', None):
    print(updated_df)

     stationId parkingBikeTotCnt  sum    stn_lat     stn_lng nearby_id  \
0        ST-10                 5    3  37.552746  126.918617   ST-2167   
1      ST-1000                10    4  37.510380  126.866798   ST-2014   
2      ST-1002                54    5  37.529900  126.876541   ST-1505   
3      ST-1003                 6    3  37.539551  126.828300   ST-2794   
4      ST-1004                 4    3  37.514099  126.831001   ST-1498   
5      ST-1005                29    3  37.513950  126.856056    ST-314   
6      ST-1006                16    3  37.536377  126.871513    ST-997   
7      ST-1007                10    2  37.522190  126.836700   ST-1500   
8      ST-1008                 3    2  37.522560  126.849480    ST-996   
9      ST-1010                10    3  37.536201  126.827797   ST-1009   
10     ST-1011                41    1  37.536369  126.831711    ST-433   
11     ST-1012                16    1  37.539520  126.825401   ST-2769   
12     ST-1013                 1    2 

In [105]:
# 없는 데이터 날리기
updated_df = updated_df.dropna(subset=['nearby_id\'s sum'])

In [106]:
with pd.option_context('display.max_rows', None):
    print(updated_df)

     stationId parkingBikeTotCnt  sum    stn_lat     stn_lng nearby_id  \
0        ST-10                 5    3  37.552746  126.918617   ST-2167   
1      ST-1000                10    4  37.510380  126.866798   ST-2014   
3      ST-1003                 6    3  37.539551  126.828300   ST-2794   
4      ST-1004                 4    3  37.514099  126.831001   ST-1498   
6      ST-1006                16    3  37.536377  126.871513    ST-997   
7      ST-1007                10    2  37.522190  126.836700   ST-1500   
8      ST-1008                 3    2  37.522560  126.849480    ST-996   
11     ST-1012                16    1  37.539520  126.825401   ST-2769   
12     ST-1013                 1    2  37.550732  126.864578    ST-528   
13     ST-1014                20    4  37.508900  126.842682    ST-317   
14     ST-1015                 3    3  37.543842  126.882545   ST-1736   
15     ST-1016                 2    2  37.522282  126.839699   ST-2770   
16     ST-1017                25    6 

In [107]:
#날씨 API에 사용할 격자X, 격자Y 구하기
df3 = pd.read_excel('기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20210401).xlsx')
df3

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,kor,5013060000,제주특별자치도,서귀포시,대천동,52,32,126,28,46.47,33,14,50.91,126.479575,33.247475,NaN
3770,kor,5013061000,제주특별자치도,서귀포시,중문동,51,32,126,26,12.40,33,14,53.44,126.436778,33.248178,NaN
3771,kor,5013062000,제주특별자치도,서귀포시,예래동,50,32,126,23,59.67,33,15,3.69,126.399908,33.251025,NaN
3772,kor,5019000000,이어도,NaN,NaN,28,8,0,0,0.00,0,0,0.00,0.000000,0.000000,NaN


In [108]:
import pandas as pd
import math
from math import cos, sin, asin, sqrt, radians

#가장 가까운 격자X, 격자Y 구하기 위해서 거리 구하기

# Haversine formula를 사용하여 두 지점 사이의 거리를 계산하는 함수
def haversine(lat1, lon1, lat2, lon2):
    # 위도, 경도를 라디안으로 변환
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # 지구의 반지름 (단위: km)
    return c * r

# updated_df의 각 행에 대해 가장 가까운 거리에 해당하는 df3의 행을 찾아 업데이트
for index, row in updated_df.iterrows():
    min_distance = None
    min_idx = None
    
    for idx, df_row in df3.iterrows():
        distance = haversine(row['stn_lat'], row['stn_lng'], df_row['위도(초/100)'], df_row['경도(초/100)'])
        if min_distance is None or distance < min_distance:
            min_distance = distance
            min_idx = idx
    
    updated_df.at[index, '행정구역코드'] = df3.at[min_idx, '행정구역코드']
    updated_df.at[index, '격자 X'] = df3.at[min_idx, '격자 X']
    updated_df.at[index, '격자 Y'] = df3.at[min_idx, '격자 Y']
    
updated_df

,stationId,parkingBikeTotCnt,sum,stn_lat,stn_lng,nearby_id,nearby_id's sum,행정구역코드,격자 X,격자 Y
0,ST-10,5,3,37.552746,126.918617,ST-2167,2.0,1.144066e+09,59.0,126.0
1,ST-1000,10,4,37.510380,126.866798,ST-2014,4.0,1.147067e+09,58.0,126.0
3,ST-1003,6,3,37.539551,126.828300,ST-2794,1.0,1.147060e+09,57.0,126.0
4,ST-1004,4,3,37.514099,126.831001,ST-1498,2.0,1.147061e+09,57.0,126.0
6,ST-1006,16,3,37.536377,126.871513,ST-997,3.0,1.150053e+09,58.0,126.0
...,...,...,...,...,...,...,...,...,...,...
2709,ST-992,17,1,37.549061,126.954178,ST-213,1.0,1.144056e+09,59.0,126.0
2710,ST-993,4,2,37.521511,126.857384,ST-2761,2.0,1.147065e+09,58.0,126.0
2712,ST-995,3,1,37.510597,126.857323,ST-1504,4.0,1.147064e+09,58.0,126.0
2713,ST-996,14,1,37.524334,126.850548,ST-1008,2.0,1.147057e+09,58.0,126.0


In [109]:
# stationId, stn_lat, stn_lng 컬럼 삭제
updated_df = updated_df.drop(['stn_lat', 'stn_lng', '행정구역코드'], axis=1)

In [110]:
updated_df

,stationId,parkingBikeTotCnt,sum,nearby_id,nearby_id's sum,격자 X,격자 Y
0,ST-10,5,3,ST-2167,2.0,59.0,126.0
1,ST-1000,10,4,ST-2014,4.0,58.0,126.0
3,ST-1003,6,3,ST-2794,1.0,57.0,126.0
4,ST-1004,4,3,ST-1498,2.0,57.0,126.0
6,ST-1006,16,3,ST-997,3.0,58.0,126.0
...,...,...,...,...,...,...,...
2709,ST-992,17,1,ST-213,1.0,59.0,126.0
2710,ST-993,4,2,ST-2761,2.0,58.0,126.0
2712,ST-995,3,1,ST-1504,4.0,58.0,126.0
2713,ST-996,14,1,ST-1008,2.0,58.0,126.0


In [111]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# 새로운 열 추가
updated_df['기온'] = 0
updated_df['강수량'] = 0
updated_df['풍속'] = 0

# 중복된 nx, ny 값을 제거하여 unique_pairs 리스트에 저장
unique_pairs = updated_df[['격자 X', '격자 Y']].drop_duplicates().values.tolist()

# API 요청을 보낼 함수
def fetch_weather_data(pair):
    nx, ny = map(int, pair)
    
    today = datetime.now() - timedelta(hours=1)
    today_date = today.strftime("%Y%m%d")
    today_hour = today.strftime("%H")+"00"
    
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    params = {
        'serviceKey': '5oSWTH+Lqi0ZbOgu/nZ23woYupdrJsK0cDwQOV99yBuBXkAcqqQE1HL0ucMNkmt0tPqBvaFPYpo/RlsZ8WjWLQ==',
        'pageNo': 1,
        'numofRows': 10,
        'dataType':'XML',
        'base_date': today_date,
        'base_time': today_hour,
        'nx': nx,
        'ny': ny
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        root = ET.fromstring(response.text)
        weather_data = {category.find('category').text: category.find('obsrValue').text 
                        for category in root.findall('.//item')}
        
        # 해당 좌표에 대한 업데이트
        mask = (updated_df['격자 X'] == nx) & (updated_df['격자 Y'] == ny)
        updated_df.loc[mask, '기온'] = weather_data.get('T1H')
        updated_df.loc[mask, '강수량'] = weather_data.get('RN1')
        updated_df.loc[mask, '풍속'] = weather_data.get('WSD')
        
    else:
        print(f"요청 실패 - {nx}, {ny}: {response.status_code}")

# 각 unique pair에 대해 API 요청 보내기
for pair in unique_pairs:
    fetch_weather_data(pair)
    


In [112]:
print(updated_df)

     stationId parkingBikeTotCnt  sum nearby_id  nearby_id's sum  격자 X   격자 Y  \
0        ST-10                 5    3   ST-2167              2.0  59.0  126.0   
1      ST-1000                10    4   ST-2014              4.0  58.0  126.0   
3      ST-1003                 6    3   ST-2794              1.0  57.0  126.0   
4      ST-1004                 4    3   ST-1498              2.0  57.0  126.0   
6      ST-1006                16    3    ST-997              3.0  58.0  126.0   
...        ...               ...  ...       ...              ...   ...    ...   
2709    ST-992                17    1    ST-213              1.0  59.0  126.0   
2710    ST-993                 4    2   ST-2761              2.0  58.0  126.0   
2712    ST-995                 3    1   ST-1504              4.0  58.0  126.0   
2713    ST-996                14    1   ST-1008              2.0  58.0  126.0   
2714    ST-997                 4    3    ST-310              3.0  58.0  126.0   

       기온 강수량   풍속  
0     

In [113]:
updated_df['기온'] = updated_df['기온'].astype(float)
updated_df['풍속'] = updated_df['풍속'].astype(float)


In [114]:
import numpy as np
def calculate_wind_chill(temperature, wind_speed):
    # 체감온도 공식 계산
    wind_chill = 13.12 + 0.6215 * temperature - 11.37 * np.power(wind_speed, 0.16) + 0.3965 * np.power(wind_speed, 0.16) * temperature
    
    return wind_chill

# 체감온도 계산 및 열 추가
updated_df['체감온도'] = calculate_wind_chill(updated_df['기온'], updated_df['풍속'])

#필요 없는 행 날리기
updated_df = updated_df.drop(['격자 X', '격자 Y', '기온', '풍속'], axis=1)

# 결과 출력
print(updated_df)

     stationId parkingBikeTotCnt  sum nearby_id  nearby_id's sum 강수량      체감온도
0        ST-10                 5    3   ST-2167              2.0   0  4.355111
1      ST-1000                10    4   ST-2014              4.0   0  5.176456
3      ST-1003                 6    3   ST-2794              1.0   0  3.379519
4      ST-1004                 4    3   ST-1498              2.0   0  3.379519
6      ST-1006                16    3    ST-997              3.0   0  5.176456
...        ...               ...  ...       ...              ...  ..       ...
2709    ST-992                17    1    ST-213              1.0   0  4.355111
2710    ST-993                 4    2   ST-2761              2.0   0  5.176456
2712    ST-995                 3    1   ST-1504              4.0   0  5.176456
2713    ST-996                14    1   ST-1008              2.0   0  5.176456
2714    ST-997                 4    3    ST-310              3.0   0  5.176456

[2244 rows x 7 columns]


In [115]:
today = datetime.now() - timedelta(hours=1)
today_date = today.strftime("%Y%m%d")
today_hour = today.strftime("%H")+"00"
today_day = (today.weekday()) + 1

result = {'borrowed_hour':str(int(today_hour[:2])), 'borrowed_day': today_day,'borrowed_date':today_date}
    
updated_df['borrowed_hour'] = result['borrowed_hour']
updated_df['borrowed_day'] = result['borrowed_day']
updated_df['borrowed_date'] = result['borrowed_date']


updated_df['is_holiday'] = 1

In [116]:
updated_df

,stationId,parkingBikeTotCnt,sum,nearby_id,nearby_id's sum,강수량,체감온도,borrowed_hour,borrowed_day,borrowed_date,is_holiday
0,ST-10,5,3,ST-2167,2.0,0,4.355111,17,7,20231203,1
1,ST-1000,10,4,ST-2014,4.0,0,5.176456,17,7,20231203,1
3,ST-1003,6,3,ST-2794,1.0,0,3.379519,17,7,20231203,1
4,ST-1004,4,3,ST-1498,2.0,0,3.379519,17,7,20231203,1
6,ST-1006,16,3,ST-997,3.0,0,5.176456,17,7,20231203,1
...,...,...,...,...,...,...,...,...,...,...,...
2709,ST-992,17,1,ST-213,1.0,0,4.355111,17,7,20231203,1
2710,ST-993,4,2,ST-2761,2.0,0,5.176456,17,7,20231203,1
2712,ST-995,3,1,ST-1504,4.0,0,5.176456,17,7,20231203,1
2713,ST-996,14,1,ST-1008,2.0,0,5.176456,17,7,20231203,1


In [117]:
updated_df.rename(columns={'parkingBikeTotCnt': 'remain_bicycle'}, inplace=True)
updated_df.rename(columns={'nearby_id\'s sum': 'borrowed_num_nearby'}, inplace=True)
updated_df.rename(columns={'강수량': '강수량(mm)'}, inplace=True)
updated_df.rename(columns={'체감온도': 'wind_chill'}, inplace=True)
updated_df

,stationId,remain_bicycle,sum,nearby_id,borrowed_num_nearby,강수량(mm),wind_chill,borrowed_hour,borrowed_day,borrowed_date,is_holiday
0,ST-10,5,3,ST-2167,2.0,0,4.355111,17,7,20231203,1
1,ST-1000,10,4,ST-2014,4.0,0,5.176456,17,7,20231203,1
3,ST-1003,6,3,ST-2794,1.0,0,3.379519,17,7,20231203,1
4,ST-1004,4,3,ST-1498,2.0,0,3.379519,17,7,20231203,1
6,ST-1006,16,3,ST-997,3.0,0,5.176456,17,7,20231203,1
...,...,...,...,...,...,...,...,...,...,...,...
2709,ST-992,17,1,ST-213,1.0,0,4.355111,17,7,20231203,1
2710,ST-993,4,2,ST-2761,2.0,0,5.176456,17,7,20231203,1
2712,ST-995,3,1,ST-1504,4.0,0,5.176456,17,7,20231203,1
2713,ST-996,14,1,ST-1008,2.0,0,5.176456,17,7,20231203,1


In [118]:
updated_df.to_csv('realdata따릉이 5-6 일요일.csv', index=False)